# Experiment Env. for Stack DIE (TD+BD+EMIB) Prototype CTM on RHSC-ET Thermal Analysis #

In [1]:
#### import packages ####
import os
import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox
from ipyfilechooser import FileChooser

#### import MY packages ####
from parsers import RHSCETparser
from setups import FCpowermap
from setups import utilities
from setups import UIdisplay

In [2]:
### Create CASE ###
CASE = widgets.Text(value="TEST", description="CASE Name", disabled=False)
display(CASE)

Text(value='TEST', description='CASE Name')

In [ ]:
## Create CTMs: Top Die (TD), Bot Die (BD) & RDL ##

In [3]:
UI, TD = UIdisplay.askPowerDensityRCfiles("Top Die")
### TD[0]: [power file, power_file_type], TD[1]: metal density file, TD[2]: RC techfile
display(UI)

In [ ]:
UI, BD = UIdisplay.askPowerDensityRCfiles("Bot Die")
display(UI)

In [ ]:
UI, RDL = UIdisplay.askPowerDensityRCfiles("RDL Die")
display(UI)

In [ ]:
### Create case in CASEs folder base on CASE Name ###
root = os.path.join(os.getcwd(), "CASEs")
caseFolder = utilities.createCASE(root, CASE.value)
caseDB = os.path.join(caseFolder, "DB")

In [ ]:
### Optional: Manually given the design area ###
import importlib
importlib.reload(UIdisplay)

_TD, TDGemo = UIdisplay.askDesignArea("Top Die")
_BD, BDGemo = UIdisplay.askDesignArea("Bot Die")
_RDL, RDLGemo = UIdisplay.askDesignArea("RDL Die")
VBox([_TD, _BD, _RDL])

In [ ]:
### Exam Power Map ###
import importlib
importlib.reload(FCpowermap)

if TDGemo[0].value in ["True"]:
    TDArea = [TDGemo[1].value, TDGemo[2].value, TDGemo[3].value, TDGemo[4].value]
else:
    TDArea = []

print("Top Die Area: {}".format(TDArea))
TDpwrView = FCpowermap.FCPowerView(TD[0][0].selected, TD[0][1].value, DesignArea=TDArea)
#TDpwrView.plot(saveImg="TopPwr.png", isShow=False)

if BDGemo[0].value in ["True"]:
    BDArea = [BDGemo[1].value, BDGemo[2].value, BDGemo[3].value, BDGemo[4].value]
else:
    BDArea = []

print("Bot Die Area: {}".format(BDArea))
BDpwrView = FCpowermap.FCPowerView(BD[0][0].selected, BD[0][1].value, DesignArea=BDArea)
#BDpwrView.plot(saveImg="BotPwr.png", isShow=False)

if RDLGemo[0].value in ["True"]:
    RDLArea = [RDLGemo[1].value, RDLGemo[2].value, RDLGemo[3].value, RDLGemo[4].value]
else:
    RDLArea = []

print("RDL Area: {}".format(RDLArea))
RDLpwrView = FCpowermap.FCPowerView(RDL[0][0].selected, RDL[0][1].value, DesignArea=RDLArea)
#RDLpwrView.plot(isShow=False)

In [ ]:
### Create Prototype CTM ###
TDArea, TDmhsPath = TDpwrView.RHSCETmhs(outputFolder=caseDB, outputName="TDpwd.mhs")
BDArea, BDmhsPath = BDpwrView.RHSCETmhs(outputFolder=caseDB, outputName="BDpwd.mhs")
RDLArea, RDLmhsPath = RDLpwrView.RHSCETmhs(outputFolder=caseDB, outputName="RDLpwd.mhs")  ## zero power

_TDCTM, TDCTM = UIdisplay.setupCTMResolution("Top Die")
_BDCTM, BDCTM = UIdisplay.setupCTMResolution("Bot Die")
_RDLCTM, RDLCTM = UIdisplay.setupCTMResolution("RDL Die")
VBox([_TDCTM, _BDCTM, _RDLCTM])

In [ ]:
#print(designArea, PWfc.selected, MDfc.selected, TFfc.selected, CTM_resolution.value)
os.system("rm -rf {}".format(os.path.join(caseFolder, "*CTM*")))
DIEsDict = {}
for DIE in [["TD", TD, TDArea, TDCTM, TDmhsPath], ["BD", BD, BDArea, BDCTM, BDmhsPath], ["RDL", RDL, RDLArea, RDLCTM, RDLmhsPath]]:
    Name, Setup, Area, CTM, mhsPath = DIE[0], DIE[1], DIE[2], DIE[3], DIE[4]
    
    DIE_X = "{:.3f}".format(Area[2]-Area[0])
    DIE_Y = "{:.3f}".format(Area[3]-Area[1])
    CTM_r = str(CTM[1].value)
    
    ### make a copy to the case DB folder ###
    os.system("cp -rf {} {}".format(Setup[0][0].selected, caseDB))
    os.system("cp -rf {} {}".format(Setup[1].selected, caseDB))
    os.system("cp -rf {} {}".format(Setup[2].selected, caseDB))

    CTMparaDict = {"RUN_DIR":caseFolder, "CTMNAME":CTM[0].value, "DIE_LENGTH":DIE_X, "DIE_WIDTH":DIE_Y, \
                   "RESOLUTION":CTM_r, "PWRMAP":mhsPath, "DENSITY":Setup[1].selected, "TOTEM_TF":Setup[2].selected}

    templateScript = os.path.join(os.path.join(os.getcwd(), "ET_scripts"), "prototypeCTM.tcl")
    ETscript = utilities.createETscript(CTMparaDict, template=templateScript, \
                                        outputFolder=caseDB, outputName="genPrototype{}CTM.tcl".format(Name))

    runcmd = ["#!/bin/csh", "rm -rf {}".format(os.path.join(caseFolder, "_CTM_et")),\
              "source /nfs/site/disks/ifs_itip_disk001/cchuang/Tools/licenses/ansys.csh", \
              "redhawk_sc_et -3dic -ng {}".format(ETscript)]

    runcshPath = os.path.join(caseDB, "gen{}CTM.csh".format(Name))
    progBar = widgets.IntProgress(max=100)
    display(progBar)
    
    CTMpath = os.path.join(os.path.join(CTMparaDict["RUN_DIR"], "CTM"), CTMparaDict["CTMNAME"]+".tar.gz")
    DIEsDict.setdefault(Name, {"CTMpath":CTMpath, "CTMdetails":CTMparaDict})
    
    utilities.submitJob(runcmd, runcshPath=runcshPath, progressBar=progBar, checkFiles=[CTMpath])

In [ ]:
### Exam CTM power profile ###
#ctmImgPath = os.path.join(os.path.join(CTMparaDict["RUN_DIR"], "CTM"), "ctm_pwr.png")
#utilities.viewImg(ctmImgPath)

In [ ]:
import importlib
importlib.reload(UIdisplay)

### Setup DIE thermal simulation inputs ###
ETCaseName = widgets.Text(value="DEMO", description="ET Project Name", disabled=False)
TD_H = widgets.FloatText(value=19.1, description="TD height", disabled=False)
BD_H = widgets.FloatText(value=17.1, description="BD height", disabled=False)
RDL_H = widgets.FloatText(value=11.0, description="RDL height", disabled=False)

totalPWR = widgets.FloatText(value=1.0, description="Total PWR", disabled=False)
deltaT = widgets.FloatText(value=10.0, description="delta T", disabled=False)

### Bump/Ball, unit: mm ###
#UB_D = widgets.FloatText(value=0.02, description="uBump diameter", disabled=False)
#UB_H = widgets.FloatText(value=0.005, description="uBump height", disabled=False)
#UB_PX = widgets.FloatText(value=0.025, description="uBump pitchX", disabled=False)
#UB_PY = widgets.FloatText(value=0.025, description="uBump pitchY", disabled=False)
#UB_AX = widgets.intText(value=5, description="uBump arrayX", disabled=False)
#UB_AY = widgets.intText(value=5, description="uBump arrayY", disabled=False)

#C4_D = widgets.FloatText(value=0.05, description="C4 diameter", disabled=False)
#C4_H = widgets.FloatText(value=0.01, description="C4 height", disabled=False)
#C4_PX = widgets.FloatText(value=0.07, description="C4 pitchX", disabled=False)
#C4_PY = widgets.FloatText(value=0.07, description="C4 pitchY", disabled=False)
#C4_AX = widgets.intText(value=5, description="C4 arrayX", disabled=False)
#C4_AY = widgets.intText(value=5, description="C4 arrayY", disabled=False)

### default: [diameter, height, pitchX, pitchY, arrayX, arrayY] ###
_uBump, uBump = UIdisplay.setupBumpBall("uBump", Gtype="ARRAY", defaultValues=[0.02, 0.005, 0.0025, 0.0025, 5, 5])
_C4, C4 = UIdisplay.setupBumpBall("C4", Gtype="ARRAY", defaultValues=[0.05, 0.01, 0.07, 0.07, 5, 5])
########################

### Layout ###
#HB1 = HBox([DIE_H, Label("um")])
#HB2 = HBox([Label("Select Thermal TF"), TTFfc, TFDIE])
#HB3 = HBox([totalPWR, Label("mW"), deltaT])
#HB4 = HBox([ambT, TOPBC, Label("H"), BOTBC, Label("H")])
VBox([ETCaseName, _uBump, _C4, HB4])

In [12]:
### Package, unit: mm ###
"""
PCB_L = widgets.FloatText(value=2.0, description="PCB length", disabled=False)
PCB_W = widgets.FloatText(value=2.0, description="PCB width", disabled=False)
MC_L = widgets.FloatText(value=0.44, description="MC length", disabled=False)
MC_W = widgets.FloatText(value=0.42, description="MC width", disabled=False)
MC_H = widgets.FloatText(value=0.0412, description="MC height", disabled=False)
TIM_L = widgets.FloatText(value=0.44, description="TIM length", disabled=False)
TIM_W = widgets.FloatText(value=0.42, description="TIM width", disabled=False)
TIM_H = widgets.FloatText(value=0.01, description="TIM height", disabled=False)
LID_L = widgets.FloatText(value=0.44, description="LID length", disabled=False)
LID_W = widgets.FloatText(value=0.42, description="LID width", disabled=False)
LID_H = widgets.FloatText(value=0.2, description="LID height", disabled=False)
"""
########################
import importlib
importlib.reload(UIdisplay)

_MC, MC = UIdisplay.setupComponent(title="Molding Compound", componentType="MC", defaultValues=[0.44, 0.42, 0.0412])
_TIM, TIM = UIdisplay.setupComponent(title="TIM", componentType="TIM", defaultValues=[0.44, 0.42, 0.01])
_LID, LID = UIdisplay.setupComponent(title="LID", componentType="LID", defaultValues=[0.44, 0.42, 0.2])
_PCB, PCB = UIdisplay.setupComponent(title="PCB", componentType="PCB", defaultValues=[2.0, 2.0])
VBox([_MC, _TIM, _LID, _PCB])

In [15]:
print(MC.length.value, LID.height.value, str(MC.height.value))

0.44 0.2 0.0412


In [14]:
TTFfc = FileChooser("./")
TDTFDIE = widgets.Text(value="1P11M_wTSV_BS4Mi", description="TTF TD Label", disabled=False)
BDTFDIE = widgets.Text(value="1P11M_wTSV_BS2Mi", description="TTF BD Label", disabled=False)
RDLTFDIE = widgets.Text(value="5RDL", description="TTF RDL Label", disabled=False)

### Conditions ###
#ambT = widgets.FloatText(value=25.0, description="Ambient T", disabled=False)
#TOPBC = widgets.FloatText(value=1000.0, description="Top BC", disabled=False)
#BOTBC = widgets.FloatText(value=10.0, description="Bot BC", disabled=False)
#SIDEBC = widgets.FloatText(value=10.0, description="Side BC", disabled=False)
import importlib
importlib.reload(UIdisplay)

_BC, BCs = UIdisplay.setupBoundarys(defaultValues=[25.0, 1000, 10, 10])
display(_BC)

In [ ]:
### simple HTC estimation: to refine the HTC values ###
refHTC = utilities.simpleCalHTC(totalPWR.value, designArea, deltaT=deltaT.value)
print("Estimated HTC: {}".format(refHTC))

In [ ]:
### Create ET script ###
DIEparaDict = {"RUN_DIR":caseFolder, "NAME":ETCaseName.value, 
               "CTMTD_PATH":, "CTMBD_PATH":, "CTMRDL_PATH":,
               "TD_LENGTH":, "TD_WIDTH":, "TD_HEIGHT":,
               "BD_LENGTH":, "BD_WIDTH":, "BD_HEIGHT":,
               "RDL_LENGTH":, "RDL_WIDTH":, "RDL_HEIGHT":,
               "PCB_LENGTH":str(PCB.length.value), "PCB_WIDTH":str(PCB.length.value), 
               "MC_LENGTH":str(MC.length.value), "MC_WIDTH":str(MC.width.value), "MC_HEIGHT":str(MC.height.value),
               "TIM_LENGTH":str(TIM.length.value), "TIM_WIDTH":str(TIM.width.value), "TIM_HEIGHT":str(TIM.height.value),
               "LID_LENGTH":str(LID.length.value), "LID_WIDTH":str(LID.width.value), "LID_HEIGHT":str(LID.height.value),
               "UBUMP_D":, "UBUMP_H":, "UBUMP_PX":, "UBUMP_PY":, "UBUMP_AX":, "UBUMP_AY":,
               "C4_D":, "C4_H":, "C4_PX":, "C4_PY":, "C4_AX":, "C4_AY":,
               "TTF":TTFfc.selected, "TFTD":, "TFBD":, "TFRDL":,
               "AMBIENT_T":str(BCs.ambT.value), "TOPBC":str(BCs.topBC.value), "BOTBC":str(BCs.botBC.value), "SIDEBC":str(BCs.sideBC.value) }

templateScript = os.path.join(os.path.join(os.getcwd(), "ET_scripts"), "stackDIE_TD+BD+EMIB.tcl")
ETscript = utilities.createETscript(DIEparaDict, template=templateScript, outputFolder=caseDB, outputName="DEMO_et.tcl")

ETFolder = os.path.join(caseFolder, ETCaseName.value+"_et")
runcmd = ["#!/bin/csh", "rm -rf {}".format(ETFolder),\
          "source /nfs/site/disks/ifs_itip_disk001/cchuang/Tools/licenses/ansys.csh", \
          "redhawk_sc_et -3dic -ng {}".format(ETscript)]
runcshPath = os.path.join(caseDB, "runET.csh")

progBar = widgets.IntProgress(max=100)
display(progBar)
TProfile = os.path.join(ETFolder, "ThermalProfile_DIE.txt")
checkFiles = [os.path.join(ETFolder, "analysis/STATIC_RUN/thermal/STATIC_RUN.dbg"), \
              TProfile]
utilities.submitJob(runcmd, runcshPath=runcshPath, progressBar=progBar, checkFiles=checkFiles)

In [ ]:
### analysis ET Thermal Profile ###
ETview = RHSCETparser.RHSCETView(TProfile)
ETview.plot(isShow=True)

In [ ]:
### Optional: Submit job to open RHSC_ET GUI mode ###
runcmd = ["#!/bin/csh",
          "source /nfs/site/disks/ifs_itip_disk001/cchuang/Tools/licenses/ansys.csh", \
          "redhawk_sc_et -3dic"]
runcshPath = os.path.join(caseDB, "runETGUI.csh")
utilities.submitJob(runcmd, runcshPath=runcshPath)

In [ ]:
import datetime

a = "3e-08"
a = float(a)
print(a, type(a))